In [1]:
!curl -L -o dns-tunneling-queries-classification.zip\
  https://www.kaggle.com/api/v1/datasets/download/saurabhshahane/dns-tunneling-queries-classification

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1529k  100 1529k    0     0  2139k      0 --:--:-- --:--:-- --:--:-- 9358k


In [2]:
!unzip dns-tunneling-queries-classification.zip

Archive:  dns-tunneling-queries-classification.zip
replace training.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [3]:
PRINTABLE = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"

assert len(PRINTABLE) == 94
print("Length:", len(PRINTABLE))

char_to_idx = {c: i for i, c in enumerate(PRINTABLE)}
char_to_idx

Length: 94


{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'a': 10,
 'b': 11,
 'c': 12,
 'd': 13,
 'e': 14,
 'f': 15,
 'g': 16,
 'h': 17,
 'i': 18,
 'j': 19,
 'k': 20,
 'l': 21,
 'm': 22,
 'n': 23,
 'o': 24,
 'p': 25,
 'q': 26,
 'r': 27,
 's': 28,
 't': 29,
 'u': 30,
 'v': 31,
 'w': 32,
 'x': 33,
 'y': 34,
 'z': 35,
 'A': 36,
 'B': 37,
 'C': 38,
 'D': 39,
 'E': 40,
 'F': 41,
 'G': 42,
 'H': 43,
 'I': 44,
 'J': 45,
 'K': 46,
 'L': 47,
 'M': 48,
 'N': 49,
 'O': 50,
 'P': 51,
 'Q': 52,
 'R': 53,
 'S': 54,
 'T': 55,
 'U': 56,
 'V': 57,
 'W': 58,
 'X': 59,
 'Y': 60,
 'Z': 61,
 '!': 62,
 '"': 63,
 '#': 64,
 '$': 65,
 '%': 66,
 '&': 67,
 "'": 68,
 '(': 69,
 ')': 70,
 '*': 71,
 '+': 72,
 ',': 73,
 '-': 74,
 '.': 75,
 '/': 76,
 ':': 77,
 ';': 78,
 '<': 79,
 '=': 80,
 '>': 81,
 '?': 82,
 '@': 83,
 '[': 84,
 '\\': 85,
 ']': 86,
 '^': 87,
 '_': 88,
 '`': 89,
 '{': 90,
 '|': 91,
 '}': 92,
 '~': 93}

In [4]:
import pandas as pd

df = pd.read_csv("training.csv", header=None, names=["label", "domain"])
df.columns

Index(['label', 'domain'], dtype='object')

In [5]:
print("labels with 1 (exfil)", (df['label'] == 1).sum())

labels with 1 (exfil) 12000


In [6]:
print("labels with 1 (benign)", (df['label'] == 0).sum())

labels with 1 (benign) 3000


In [7]:
import numpy as np

def domain_to_features(domain: str) -> np.ndarray:
  domain = domain.lower()
  counts = np.zeros(len(PRINTABLE), dtype=np.float32)
  for c in domain:
    if c in char_to_idx:
      counts[char_to_idx[c]] += 1

  length = len(domain)
  if length == 0:
    entropy = 0.0
  else:
    probs = counts / length
    entropy = -np.sum(probs * np.log2(probs + 1e-12)) # Note to self, the addition here avoids log2(0) which is undefined

  entropy_avg = entropy
  len_avg = float(length)

  return np.concatenate([counts, [length, entropy, entropy_avg, len_avg]])

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Note this is a recreation based on the information i could gather from this blog post: https://www.splunk.com/en_us/blog/security/machine-learning-in-security-detect-dns-data-exfiltration-using-deep-learning.html

class SplunkDNSExfilModel(nn.Module):
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.dropout(x, p=0.5, training=self.training)
    x = F.relu(self.fc2(x))
    x = F.dropout(x, p=0.5, training=self.training)
    x = self.fc3(x)
    return x.squeeze(1) # remove a dim since it returns shape (256, 1), will now return (256)

  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(98, 256)
    self.fc2 = nn.Linear(256, 256)
    self.fc3 = nn.Linear(256, 1)


In [9]:
X = df['domain'].values
y = df['label'].values
X, y

(array(['q+Z8AnwaBA.hidemyself.org.', 'q+Z8A3wbBA.hidemyself.org.',
        'q+Z8BHwcBA.hidemyself.org.', ..., 'r17801.tunnel.tuns.org.',
        'dIUEABDEGSBAAAQAGT7HQUAAAAMFAAAAC37CAAFUWW5XBOHRPM4PIAEAAHT6M2A.AAAEAQQCWZBIH6BEGV33RX3C4CKGEOPSNLKQNAMMKV4EPKKFENMLTM2O6BLOES2.DXNVSRGEDZRWYPR3UFL5HUBRVY5OYVOM4VA2N3LZ44KDGBXB4TMKEY767FHCKFL.MPBXZUCIGNPOCNEHCZVK62CHBS6KYI4J4TR2.tunnel.tuns.org.',
        'r17802.tunnel.tuns.org.'], shape=(15000,), dtype=object),
 array([1, 1, 1, ..., 1, 1, 1], shape=(15000,)))

In [10]:
from torch.utils.data import Dataset, DataLoader
class DNSDataset(Dataset):
  def __init__(self, domains, labels):
    self.features = torch.from_numpy(np.stack([domain_to_features(domain) for domain in domains])).float()
    self.labels = torch.from_numpy(labels.astype(np.float32))

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

In [11]:
train_dataset = DNSDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [12]:
df = pd.read_csv("validating.csv", header=None, names=["label", "domain"])
X_validation = df['domain'].values
y_validation = df['label'].values
X, y

(array(['q+Z8AnwaBA.hidemyself.org.', 'q+Z8A3wbBA.hidemyself.org.',
        'q+Z8BHwcBA.hidemyself.org.', ..., 'r17801.tunnel.tuns.org.',
        'dIUEABDEGSBAAAQAGT7HQUAAAAMFAAAAC37CAAFUWW5XBOHRPM4PIAEAAHT6M2A.AAAEAQQCWZBIH6BEGV33RX3C4CKGEOPSNLKQNAMMKV4EPKKFENMLTM2O6BLOES2.DXNVSRGEDZRWYPR3UFL5HUBRVY5OYVOM4VA2N3LZ44KDGBXB4TMKEY767FHCKFL.MPBXZUCIGNPOCNEHCZVK62CHBS6KYI4J4TR2.tunnel.tuns.org.',
        'r17802.tunnel.tuns.org.'], shape=(15000,), dtype=object),
 array([1, 1, 1, ..., 1, 1, 1], shape=(15000,)))

In [13]:
print("labels with 1 (exfil)", (df['label'] == 1).sum())
print("labels with 0 (benign)", (df['label'] == 0).sum())

labels with 1 (exfil) 4000
labels with 0 (benign) 1000


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_validation, y_validation, test_size=0.5, random_state=42)

In [15]:
validation_dataset = DNSDataset(X_train, y_train)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

test_dataset = DNSDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [16]:
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([0.25]))

In [17]:
model = SplunkDNSExfilModel()

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from sklearn.metrics import roc_auc_score

epochs = 2
best_val_auc = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
  for xb, yb in train_loader:
    model.train()
    xb, yb = xb.to(device), yb.to(device)
    pred = model(xb)
    loss = criterion(pred, yb)
    print(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  model.eval()
  with torch.no_grad():
    val_preds = torch.sigmoid(model(validation_dataset.features.to(device))).cpu().numpy().flatten()
    val_auc = roc_auc_score(validation_dataset.labels, val_preds)
    print(val_auc)

    if val_auc > best_val_auc:
      best_val_auc = val_auc
      torch.save(model.state_dict(), "splunk_dns_exfil_mlp_best.pth")

    if (epoch+1) % 10 == 0:
          print(f"Epoch {epoch+1:02d} | Val AUC: {val_auc:.6f}")

tensor(0.3662, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9807679849706864
tensor(0.1429, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9912342499919023
tensor(0.2093, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9915308279078807
tensor(0.3052, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9918101982314643
tensor(0.2105, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9924539646292878
tensor(0.1852, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9935309574709292
tensor(0.1848, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9954602322417646
tensor(0.1462, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9977498542415704
tensor(0.1660, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9979522965050367
tensor(0.1929, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.997492752566968
tensor(0.1714, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.999016130599553
tensor(0.1175, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
0.9993764778285233
tensor

In [20]:
model.load_state_dict(torch.load("splunk_dns_exfil_mlp_best.pth"))
model.eval()
with torch.no_grad():
    test_probs = torch.sigmoid(model(test_dataset.features.to(device))).cpu().numpy().flatten()
    test_preds = (test_probs >= 0.5)

    from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
    print("AUC:", roc_auc_score(y_test, test_probs))
    print(confusion_matrix(y_test, test_preds))
    print(classification_report(y_test, test_preds))

AUC: 1.0
[[ 494   14]
 [   0 1992]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.99       508
           1       0.99      1.00      1.00      1992

    accuracy                           0.99      2500
   macro avg       1.00      0.99      0.99      2500
weighted avg       0.99      0.99      0.99      2500



In [21]:
df

,label,domain
0,1,q+aJ04nrBA.hidemyself.org.
1,1,q+aJ1InsBA.hidemyself.org.
2,1,q+aJ1YntBA.hidemyself.org.
3,1,q+aJ1onuCDT2aA0NEkzbTnzRPgWJE7aYtQhjcSmnj74ufU...
4,1,q+aJ14nvCHKAcMCkiIhCb3eY1s8cgWt39jSJuK4299f/S5...
...,...,...
4995,1,r53953.tunnel.tuns.org.
4996,1,r53954.tunnel.tuns.org.
4997,1,r53955.tunnel.tuns.org.
4998,1,r53956.tunnel.tuns.org.


In [22]:
# base64 encoded secret aSdtIHNlY3JldCBwYXNzd29yZCBkYXRhIG13ZWhlaGVoZWhlaGU=

unseen_test_domain = "dGhpcy1pcwBtZXNzYWdlAAo.examplec2.com"
feature_vector = domain_to_features(unseen_test_domain)

model.eval()

with torch.no_grad():
  x = torch.from_numpy(feature_vector).float().unsqueeze(0).to(device)
  logit = model(x)
  prob = torch.sigmoid(logit).item()
  print(prob)

0.7332296967506409


In [23]:
unseen_test_domain = "hideinfo.org"
feature_vector = domain_to_features(unseen_test_domain)

model.eval()

with torch.no_grad():
  x = torch.from_numpy(feature_vector).float().unsqueeze(0).to(device)
  logit = model(x)
  prob = torch.sigmoid(logit).item()
  print(prob)

0.04225701466202736
